# <center> Encoding Data
## <center> SYSE 549: Secure Vehicle and Industrial Networking
## <center><img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="400" /> 
### <center> Instructor: Dr. Jeremy Daily

## Lesson Outcomes
After completing this exercise, students should be able to:
1. Realize encoded data is not encrypted.
2. Carry-out data encoding for integers of different lengths.
3. Present data as text strings or numbers based on the desired encoding.
4. Develop Python programming skills to work with different types of encoding.

### Overview
1. Define a string of bytes
2. Explore integer encoding
3. Learn the struct library
4. Understand how to encode data as text with different codecs
5. Represent binary only using text using base64 encoding

In [1]:
# Given a series of bits
# This could represent some data frame in a CAN message
a = 0b0110000101110100011101000110000101100011011010110000110100001010

In [2]:
#What is a?
# The python default is an integer
print(a)
type(a)

7022365680606055690


int

In [3]:
# display integer as hex characters
print("{:016X}".format(a))

61747461636B0D0A


In [5]:
#display integer as binary
print("{:064b}".format(a))

0110000101110100011101000110000101100011011010110000110100001010


In [8]:
# Use format strings (f)
f"{a:064b}"

'0110000101110100011101000110000101100011011010110000110100001010'

In [9]:
# Python 3 also has a data type of bytes. 
# Most network traffic arrives as bytes
b = a.to_bytes(8,'big')
print(b)
type(b)

b'attack\r\n'


bytes

In [10]:
len(b)

8

In [11]:
# You can iterate through an array of bytes
for i in b:
    print(i,end = ' ')

97 116 116 97 99 107 13 10 

In [18]:
# Let's print the hex characters
for i in b:
    print("{:02X}".format(i),end = ' ')

61 74 74 61 63 6B 0D 0A 

In [19]:
#Default rendering
b.hex()

'61747461636b0d0a'

In [13]:
# Make a nice display of hex with raw bytes
# This uses an efficient coding concept called list comprehension
" ".join(["{:02X}".format(i) for i in b])

'61 74 74 61 63 6B 0D 0A'

In [14]:
#Make a nice display of binary with raw bytes
" ".join(["{:08b}".format(i) for i in b])

'01100001 01110100 01110100 01100001 01100011 01101011 00001101 00001010'

In [ ]:
# pretend the last 4 bytes is a source IP address. 
# Let's display the address
print("IP: " + ".".join(["{:}".format(i) for i in b[-4:]]))

In [16]:
# Let's pretend the first four bytes are the destination IP address. 
# In Wireshark, this IP address would show up as hex:
" ".join(["{:02X}".format(i) for i in b[:4]])

'61 74 74 61'

In [21]:
# Convert hex to integer
int(b.hex(),16)

7022365680606055690

In [22]:
a

7022365680606055690

### Decoding Options
There are many options for decoding the raw bytes. The struct module is most helpful.

https://docs.python.org/3.8/library/struct.html

In [23]:
import struct

In [24]:
b

b'attack\r\n'

In [25]:
# We might have 8 single byte, unsigned integers
struct.unpack("BBBBBBBB",b)

(97, 116, 116, 97, 99, 107, 13, 10)

In [26]:
# We might have 8 single byte, signed integers
(97, 116, 116, 97, 99, 107, 13, 10)

(97, 116, 116, 97, 99, 107, 13, 10)

In [27]:
# These are the same. Let's try another example
# The \x escape tells that this is a hex string
c = b'\xDA'
print(c)

b'\xda'


In [28]:
# Unsigned Integer
struct.unpack('B',c)

(218,)

In [29]:
# Signed Integer
struct.unpack('b',c)

(-38,)

In [30]:
# The return value is always a tuple.
type(struct.unpack('b',c))

tuple

In [31]:
# To get an ingeger, index the tuple
struct.unpack('b',c)[0]

-38

In [32]:
type(struct.unpack('b',c)[0])

int

In [33]:
#What if we had 4 16-bit unsigned numbers?
struct.unpack("4H",b)

(29793, 24948, 27491, 2573)

In [34]:
# But the byte order matters,
struct.unpack("<HHHH",b)

(29793, 24948, 27491, 2573)

In [35]:
struct.unpack(">HHHH",b)

(24948, 29793, 25451, 3338)

In [36]:
#Big endian (the way humans read)
(24948).to_bytes(2,'big')

b'at'

In [37]:
#Big endian (Motorola Format)
struct.pack(">H",24948)

b'at'

In [38]:
#little endian (Reverse Byte order)
(24948).to_bytes(2,'little')

b'ta'

In [39]:
#Little endian (Intel Format)
# Note 24948 = 0x6174 
d = struct.pack("<H",0x6174)
print(d)

b'ta'


In [40]:
# Notice the reverse byte order (Little Endian or Intel)
" ".join(["{:02X}".format(i) for i in d])

'74 61'

In [41]:
# Signed 2-byte integers
struct.unpack(">h",b'\xda\x55')

(-9643,)

In [42]:
# Signed 2-byte integers
struct.unpack("<h",b'\xda\x55')

(21978,)

In [43]:
# notice the reverse byte order
print(0x55da)

21978


### Characters

In [44]:
# We might have 8 single characters
e = struct.unpack("8c",b)
print(e)

(b'a', b't', b't', b'a', b'c', b'k', b'\r', b'\n')


In [45]:
#Combine the bytes into a single string of bytes.
b''.join(e)

b'attack\r\n'

In [46]:
#This is the same as packing
f = struct.pack("cccccccc",b'a', b't', b't', b'a', b'c', b'k', b'\r', b'\n')
print(f)

b'attack\r\n'


In [47]:
# Using s enables the creation of a byte string without join
f = struct.pack("cccccccc",b'a', b't', b't', b'a', b'c', b'k', b'\r', b'\n')
print(f)

b'attack\r\n'


In [48]:
type(f)

bytes

In [49]:
#Convert to a string
str(f)

"b'attack\\r\\n'"

In [50]:
# Not really what we want. We know this is ascii text
f.decode('ascii')

'attack\r\n'

In [51]:
# Notice the extra line from the carriage return \r and new line \n.
print(f.decode('ascii'))

attack



In [52]:
# More modern is utf-8
f.decode('utf-8')

'attack\r\n'

In [53]:
# There are many character sets
f.decode('latin-1')

'attack\r\n'

In [54]:
b'attack\xc4\xfa'.decode('latin-1')

'attackÄú'

In [55]:
b'attack\xc4\xfa'.decode('ascii')

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc4 in position 6: ordinal not in range(128)

In [57]:
# We can ignore some of the non-ascii characters
b'attack\xc4\xfa'.decode('ascii','ignore')

'attack'

In [58]:
b'attack\xc4\xfa'.decode('utf-8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc4 in position 6: invalid continuation byte

In [59]:
# This keeps from getting errors
b'attack\xc4\xfa'.decode('utf-8','replace')

'attack��'

In [60]:
# Here's a valid UTF string. It uses 2 bytes.
b'attack\xc4\x8a'.decode('utf-8','strict')

'attackĊ'

### Long Integers

In [61]:
#In little-endian
struct.unpack('>LL',b)

(1635021921, 1667960074)

In [62]:
2**32

4294967296

In [63]:
#In little-endian
struct.unpack('<LL',b)

(1635021921, 168651619)

In [64]:
# Notice the symmetry on the first four bytes; atta is a palindrome
# Lets look at the last bytes
struct.unpack('<L',b'ck\r\n')

(168651619,)

In [65]:
#In big endian:
struct.unpack('>L',b'ck\r\n')

(1667960074,)

In [66]:
# The pack operation
struct.pack("<L",168651619)

b'ck\r\n'

In [67]:
#Hex as an int
0x0A0D6B63

168651619

In [68]:
# Reverse byte order
struct.pack(">L",0x0A0D6B63)

b'\n\rkc'

In [69]:
#Signed long integers
struct.unpack('>ll',b)

(1635021921, 1667960074)

### Practical Example: Decoding Vehicle Miles
SAE J1939 has a message defined as PGN 65248: Vehicle Distance. It has two 32-bit integers in the 8 byte message. The first four bytes are SPN 244: Trip Distance and the second number is SPN 245: Total Vehicle Distance, or the Odometer reading. It is represented as the number of 0.125 km that have accumulated. This message can be found in many truck log files.

In [70]:
#First, print the PGN in hex
"{:X}".format(65248)

'FEE0'

In [71]:
# A CAN log file from a truck has this line corresponding to PGN 65248
log_text = "(012.102753)  can1  18FEE000   [8] 73 49 03 00 BC E0 33 00"

In [72]:
# Parse the line into a list:
entries = log_text.split()
entries

['(012.102753)',
 'can1',
 '18FEE000',
 '[8]',
 '73',
 '49',
 '03',
 '00',
 'BC',
 'E0',
 '33',
 '00']

In [73]:
#Convert to bytes from a long string of hex characters
data_bytes = bytes.fromhex(''.join(entries[-8:]))
data_bytes

b'sI\x03\x00\xbc\xe03\x00'

In [74]:
# another way is to convert a list of integers
data_bytes = bytes([int(i,16) for i in entries[-8:]])
data_bytes

b'sI\x03\x00\xbc\xe03\x00'

In [75]:
# another way is to build the bytes iteratively. (not prefered)
data_bytes = b''
for i in entries[4:12]:
    data_bytes += int(i,16).to_bytes(1,'big')
data_bytes

b'sI\x03\x00\xbc\xe03\x00'

In [76]:
# J1939 is in little endian (Intel) format
pgn_values = struct.unpack('<LL',data_bytes)
pgn_values

(215411, 3399868)

In [77]:
# Compute mileage
SPN245 = 0.125*pgn_values[1]/1.6071
print("The Total Vehicle Distance is {:0,.1f} miles.".format(SPN245))

The Total Vehicle Distance is 264,441.2 miles.


In [78]:
# The long way
# Multiply the bytes by their place holder
value = 0
value += data_bytes[4]
value += data_bytes[5]*256
value += data_bytes[6]*256*256
value += data_bytes[7]*256*256*256
value

3399868

### 64-bit numbers

In [79]:
#Convert the double long integer a into bytes
struct.pack('>Q',a)

b'attack\r\n'

In [80]:
# See how to convert back into a 64-bit integer
struct.unpack('>Q',b)

(7022365680606055690,)

In [81]:
#Byte order is important
struct.unpack('<Q',b)

(724353189657474145,)

In [82]:
# Signed 64-bit integers
struct.unpack('>q',b)

(7022365680606055690,)

In [83]:
# The first bit must be set to get a negative number
neg_num = struct.unpack('>q',b'\xd0ttack\r\n')
neg_num

(-3425985454893495030,)

In [84]:
# We can look at this as two floats
struct.unpack('>ff',b)

(2.8183697112289195e+20, 4.335924420794005e+21)

In [85]:
# We can look at this as two floats (endianness does matter when its not a palidrome) 
struct.unpack('<ff',b)

(2.8183697112289195e+20, 6.809100250964041e-33)

In [86]:
# We can look at all 8-bytes as a double float
struct.unpack('>d',b)[0]

2.8757353661668934e+161

In [87]:
# We can look at all 8-bytes as a double float
struct.unpack('<d',b)[0]

2.989708374342575e-260

In [88]:
# Reversing byte order is not the inverse
1/struct.unpack('>d',b)[0]

3.4773714291134986e-162

## Sending bytes as text only
Base64 encoding

https://docs.python.org/3.8/library/base64.html


This is how to send cryptographic bytes in e-mail.

In [89]:
import base64

In [90]:
#As bytes
g = base64.b64encode(b)
print(g)

b'YXR0YWNrDQo='


In [91]:
base64.b64decode(g)

b'attack\r\n'

In [92]:
#As a string
g.decode('utf-8')

'YXR0YWNrDQo='

In [93]:
len(g)

12

In [94]:
len(b)

8

While the length of the data increases by 1.5, it enables transmission by email or http. Base64 encoding is very common for storing cryptographic data.

In [95]:
# Recall
b

b'attack\r\n'

In [96]:
#What about just converting to hex characters?
h = b.hex()
h

'61747461636b0d0a'

In [97]:
#Converting to printable hex doubles the length. Therefore, base64 encoding is more efficient.
len(h)

16

In [98]:
# recall: display integer as hex characters
print("{:016X}".format(a))

61747461636B0D0A


In [99]:
#Decode
base64.b64decode('YXR0YWNrDQo=')

b'attack\r\n'

In [100]:
# Generate a list of all byte values
char_list = [struct.pack("B",i) for i in range(256)]
print(char_list)

[b'\x00', b'\x01', b'\x02', b'\x03', b'\x04', b'\x05', b'\x06', b'\x07', b'\x08', b'\t', b'\n', b'\x0b', b'\x0c', b'\r', b'\x0e', b'\x0f', b'\x10', b'\x11', b'\x12', b'\x13', b'\x14', b'\x15', b'\x16', b'\x17', b'\x18', b'\x19', b'\x1a', b'\x1b', b'\x1c', b'\x1d', b'\x1e', b'\x1f', b' ', b'!', b'"', b'#', b'$', b'%', b'&', b"'", b'(', b')', b'*', b'+', b',', b'-', b'.', b'/', b'0', b'1', b'2', b'3', b'4', b'5', b'6', b'7', b'8', b'9', b':', b';', b'<', b'=', b'>', b'?', b'@', b'A', b'B', b'C', b'D', b'E', b'F', b'G', b'H', b'I', b'J', b'K', b'L', b'M', b'N', b'O', b'P', b'Q', b'R', b'S', b'T', b'U', b'V', b'W', b'X', b'Y', b'Z', b'[', b'\\', b']', b'^', b'_', b'`', b'a', b'b', b'c', b'd', b'e', b'f', b'g', b'h', b'i', b'j', b'k', b'l', b'm', b'n', b'o', b'p', b'q', b'r', b's', b't', b'u', b'v', b'w', b'x', b'y', b'z', b'{', b'|', b'}', b'~', b'\x7f', b'\x80', b'\x81', b'\x82', b'\x83', b'\x84', b'\x85', b'\x86', b'\x87', b'\x88', b'\x89', b'\x8a', b'\x8b', b'\x8c', b'\x8d', b'\x8e', b'

In [101]:
#the alphabet in base64:
j = base64.b64encode(b''.join(char_list))
print(j)

b'AAECAwQFBgcICQoLDA0ODxAREhMUFRYXGBkaGxwdHh8gISIjJCUmJygpKissLS4vMDEyMzQ1Njc4OTo7PD0+P0BBQkNERUZHSElKS0xNTk9QUVJTVFVWV1hZWltcXV5fYGFiY2RlZmdoaWprbG1ub3BxcnN0dXZ3eHl6e3x9fn+AgYKDhIWGh4iJiouMjY6PkJGSk5SVlpeYmZqbnJ2en6ChoqOkpaanqKmqq6ytrq+wsbKztLW2t7i5uru8vb6/wMHCw8TFxsfIycrLzM3Oz9DR0tPU1dbX2Nna29zd3t/g4eLj5OXm5+jp6uvs7e7v8PHy8/T19vf4+fr7/P3+/w=='


In [102]:
len(j)

344

In [103]:
len(j)/len(char_list)

1.34375

In [104]:
len(char_list)

256

Note: Base64 encoded data is NOT encrypted. No additional information is needed to decode the data. There is no key.

## Crude Ciphers
### Simple XOR encryption
Given use an XOR operation to encrypt and decrypt.

In [105]:
plain_text =  "Fourscore and seven years ago our fathers brought forth, on this continent, a new nation, conceived in liberty, and dedicated to the proposition that all men are created equal. Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived, and so dedicated, can long endure."

In [106]:
key = 5
struct.pack('B',key)

b'\x05'

In [107]:
#convert to a bytearray
plain_bytes = bytes(plain_text,'utf-8')
print(plain_bytes)

b'Fourscore and seven years ago our fathers brought forth, on this continent, a new nation, conceived in liberty, and dedicated to the proposition that all men are created equal. Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived, and so dedicated, can long endure.'


In [108]:
cipher_bytes = bytes(x ^ key for x in plain_bytes)
cipher_bytes

b'Cjpwvfjw`%dka%v`s`k%|`dwv%dbj%jpw%cdqm`wv%gwjpbmq%cjwqm)%jk%qmlv%fjkqlk`kq)%d%k`r%kdqljk)%fjkf`ls`a%lk%ilg`wq|)%dka%a`alfdq`a%qj%qm`%uwjujvlqljk%qmdq%dii%h`k%dw`%fw`dq`a%`tpdi+%Kjr%r`%dw`%`kbdb`a%lk%d%bw`dq%flsli%rdw)%q`vqlkb%rm`qm`w%qmdq%kdqljk)%jw%dk|%kdqljk%vj%fjkf`ls`a)%dka%vj%a`alfdq`a)%fdk%ijkb%`kapw`+'

In [109]:
#decrypt is the same process:
bytes(x ^ key for x in cipher_bytes)

b'Fourscore and seven years ago our fathers brought forth, on this continent, a new nation, conceived in liberty, and dedicated to the proposition that all men are created equal. Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived, and so dedicated, can long endure.'

In [110]:
#There are certain valid ranges for ascii text
# The space character is the smallest
min(plain_bytes)

32

In [111]:
#show the space
struct.pack("B",min(plain_bytes))

b' '

In [112]:
# What if you don't know the key?
for k in range(256):
    canidate_bytes = bytearray(x ^ k for x in cipher_bytes)
    if min(canidate_bytes) >= 32 and max(canidate_bytes) < 127: # then Ascii       
        print(k)
        print(canidate_bytes)
        print()

0
bytearray(b'Cjpwvfjw`%dka%v`s`k%|`dwv%dbj%jpw%cdqm`wv%gwjpbmq%cjwqm)%jk%qmlv%fjkqlk`kq)%d%k`r%kdqljk)%fjkf`ls`a%lk%ilg`wq|)%dka%a`alfdq`a%qj%qm`%uwjujvlqljk%qmdq%dii%h`k%dw`%fw`dq`a%`tpdi+%Kjr%r`%dw`%`kbdb`a%lk%d%bw`dq%flsli%rdw)%q`vqlkb%rm`qm`w%qmdq%kdqljk)%jw%dk|%kdqljk%vj%fjkf`ls`a)%dka%vj%a`alfdq`a)%fdk%ijkb%`kapw`+')

1
bytearray(b'Bkqvwgkva$ej`$waraj$}aevw$eck$kqv$beplavw$fvkqclp$bkvpl($kj$plmw$gkjpmjajp($e$jas$jepmkj($gkjgamra`$mj$hmfavp}($ej`$`a`mgepa`$pk$pla$tvktkwmpmkj$plep$ehh$iaj$eva$gvaepa`$auqeh*$Jks$sa$eva$ajceca`$mj$e$cvaep$gmrmh$sev($pawpmjc$slaplav$plep$jepmkj($kv$ej}$jepmkj$wk$gkjgamra`($ej`$wk$`a`mgepa`($gej$hkjc$aj`qva*')

2
bytearray(b"Ahrutdhub\'fic\'tbqbi\'~bfut\'f`h\'hru\'afsobut\'euhr`os\'ahuso+\'hi\'sont\'dhisnibis+\'f\'ibp\'ifsnhi+\'dhidbnqbc\'ni\'knebus~+\'fic\'cbcndfsbc\'sh\'sob\'wuhwhtnsnhi\'sofs\'fkk\'jbi\'fub\'dubfsbc\'bvrfk)\'Ihp\'pb\'fub\'bi`f`bc\'ni\'f\'`ubfs\'dnqnk\'pfu+\'sbtsni`\'pobsobu\'sofs\'ifsnhi+\'hu\'fi~\'ifsnhi\'th\'dhidbnqbc+\'fic\'th\'c

In [113]:
# A Ceasar Shift Cipher
shifted_text = bytes((x + key) for x in plain_bytes)
shifted_text

b'Ktzwxhtwj%fsi%xj{js%~jfwx%flt%tzw%kfymjwx%gwtzlmy%ktwym1%ts%ymnx%htsynsjsy1%f%sj|%sfynts1%htshjn{ji%ns%qngjwy~1%fsi%ijinhfyji%yt%ymj%uwtutxnynts%ymfy%fqq%rjs%fwj%hwjfyji%jvzfq3%St|%|j%fwj%jslflji%ns%f%lwjfy%hn{nq%|fw1%yjxynsl%|mjymjw%ymfy%sfynts1%tw%fs~%sfynts%xt%htshjn{ji1%fsi%xt%ijinhfyji1%hfs%qtsl%jsizwj3'

In [114]:
# A Ceasar Shift Cipher decipher
bytes((x - key) for x in shifted_text)

b'Fourscore and seven years ago our fathers brought forth, on this continent, a new nation, conceived in liberty, and dedicated to the proposition that all men are created equal. Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived, and so dedicated, can long endure.'

In [115]:
#Compute the key based on frequency
#spaces are frequent, so
space_guess = ord('%')
print(space_guess)
space = ord(' ')
print(space)
key_guess = space_guess - space
print(key_guess)

37
32
5


In [116]:
# A classic ceaser shift cipher with a shift of 13 (half the alphabet)
import codecs
codecs.encode('aAbcdez', 'rot13',)

'nNopqrm'

#### Examine different character encodings

In [117]:
#UTF-8
" ".join([struct.pack('B',x).decode('utf-8','ignore') for x in range(0xff)])

'\x00 \x01 \x02 \x03 \x04 \x05 \x06 \x07 \x08 \t \n \x0b \x0c \r \x0e \x0f \x10 \x11 \x12 \x13 \x14 \x15 \x16 \x17 \x18 \x19 \x1a \x1b \x1c \x1d \x1e \x1f   ! " # $ % & \' ( ) * + , - . / 0 1 2 3 4 5 6 7 8 9 : ; < = > ? @ A B C D E F G H I J K L M N O P Q R S T U V W X Y Z [ \\ ] ^ _ ` a b c d e f g h i j k l m n o p q r s t u v w x y z { | } ~ \x7f                                                                                                                               '

In [118]:
#Latin-1
" ".join([struct.pack('B',x).decode('latin-1','ignore') for x in range(0xff)])

'\x00 \x01 \x02 \x03 \x04 \x05 \x06 \x07 \x08 \t \n \x0b \x0c \r \x0e \x0f \x10 \x11 \x12 \x13 \x14 \x15 \x16 \x17 \x18 \x19 \x1a \x1b \x1c \x1d \x1e \x1f   ! " # $ % & \' ( ) * + , - . / 0 1 2 3 4 5 6 7 8 9 : ; < = > ? @ A B C D E F G H I J K L M N O P Q R S T U V W X Y Z [ \\ ] ^ _ ` a b c d e f g h i j k l m n o p q r s t u v w x y z { | } ~ \x7f \x80 \x81 \x82 \x83 \x84 \x85 \x86 \x87 \x88 \x89 \x8a \x8b \x8c \x8d \x8e \x8f \x90 \x91 \x92 \x93 \x94 \x95 \x96 \x97 \x98 \x99 \x9a \x9b \x9c \x9d \x9e \x9f \xa0 ¡ ¢ £ ¤ ¥ ¦ § ¨ © ª « ¬ \xad ® ¯ ° ± ² ³ ´ µ ¶ · ¸ ¹ º » ¼ ½ ¾ ¿ À Á Â Ã Ä Å Æ Ç È É Ê Ë Ì Í Î Ï Ð Ñ Ò Ó Ô Õ Ö × Ø Ù Ú Û Ü Ý Þ ß à á â ã ä å æ ç è é ê ë ì í î ï ð ñ ò ó ô õ ö ÷ ø ù ú û ü ý þ'

In [119]:
[" ".join([struct.pack('B',x).decode('greek','ignore') for x in range(0xff)])]

['\x00 \x01 \x02 \x03 \x04 \x05 \x06 \x07 \x08 \t \n \x0b \x0c \r \x0e \x0f \x10 \x11 \x12 \x13 \x14 \x15 \x16 \x17 \x18 \x19 \x1a \x1b \x1c \x1d \x1e \x1f   ! " # $ % & \' ( ) * + , - . / 0 1 2 3 4 5 6 7 8 9 : ; < = > ? @ A B C D E F G H I J K L M N O P Q R S T U V W X Y Z [ \\ ] ^ _ ` a b c d e f g h i j k l m n o p q r s t u v w x y z { | } ~ \x7f \x80 \x81 \x82 \x83 \x84 \x85 \x86 \x87 \x88 \x89 \x8a \x8b \x8c \x8d \x8e \x8f \x90 \x91 \x92 \x93 \x94 \x95 \x96 \x97 \x98 \x99 \x9a \x9b \x9c \x9d \x9e \x9f \xa0 ‘ ’ £ € ₯ ¦ § ¨ © ͺ « ¬ \xad  ― ° ± ² ³ ΄ ΅ Ά · Έ Ή Ί » Ό ½ Ύ Ώ ΐ Α Β Γ Δ Ε Ζ Η Θ Ι Κ Λ Μ Ν Ξ Ο Π Ρ  Σ Τ Υ Φ Χ Ψ Ω Ϊ Ϋ ά έ ή ί ΰ α β γ δ ε ζ η θ ι κ λ μ ν ξ ο π ρ ς σ τ υ φ χ ψ ω ϊ ϋ ό ύ ώ']

## Concluding Remarks
* You should see and appreciate the different ways binary data can be encoded. 
* Communications rely heavily on common codecs. 
* Encoding is not encrypting.